<a href="https://colab.research.google.com/github/edyoda/AI-Agent-Development-GENAIMD240525/blob/main/Session_4_Advanced_AutoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selector Group Chat
Key features include:

*   Model-based speaker selection
*   Configurable participant roles and descriptions
*   Prevention of consecutive turns by the same speaker (optional)
*   Customizable selection prompting
*   Customizable selection function to override the default model-based selection
*   Customizable candidate function to narrow-down the set of agents for selection using model
















In [ ]:
!pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [ ]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100


In [ ]:
from google.colab import userdata

model_client = OpenAIChatCompletionClient(
        model="deepseek-chat",
        temperature=0.85,
        api_key=userdata.get('DEEPSEEK_API_KEY'),
        base_url="https://api.deepseek.com/v1",
        model_info={
            "model_name": "deepseek-chat",
            "api_type": "openai",
            "vendor": "deepseek",
            "json_output": False,
            "vision": False,
            "function_calling": True,
            "function_call": False,
            "streaming": True,
            "cost_per_token": 0,
            "family": "deepseek",
        }
    )


/usr/local/lib/python3.11/dist-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [ ]:


planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)


In [ ]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


In [ ]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


In [ ]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)


In [ ]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [ ]:
# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
Here's the plan to answer your question:

1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent : Find the total rebounds for this player in the 2007-2008 season.
3. WebSearchAgent : Find the total rebounds for this player in the 2008-2009 season.
4. DataAnalystAgent : Calculate the percentage change in rebounds between the 2007-2008 and 2008-2009 seasons for this player.

After we receive all this information, I'll summarize the findings and terminate. 

Proceeding with step 1...
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_0_8b0e37bd-1461-4a5e-a925-2040b592e195', arguments='{"query":"Miami Heat player with the highest po

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=152, completion_tokens=136), metadata={}, content="Here's the plan to answer your question:\n\n1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchAgent : Find the total rebounds for this player in the 2007-2008 season.\n3. WebSearchAgent : Find the total rebounds for this player in the 2008-2009 season.\n4. DataAnalystAgent : Calculate the percentage change in rebounds between the 2007-2008 and 2008-2009 seasons for this player.\n\nAfter we receive all this information, I'll summarize the findings and terminate. \n\nProceeding with step 1...", type='TextMessage'), To

# Selector Function

In [ ]:
def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
Here's the plan to answer your question:

1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 NBA season
2. WebSearchAgent : Find the total rebounds for this player in the 2007-2008 season
3. WebSearchAgent : Find the total rebounds for this player in the 2008-2009 season
4. DataAnalystAgent : Calculate the percentage change in rebounds between 2007-2008 and 2008-2009 seasons

Once we have all this information, I'll summarize the findings and terminate. 

Would you like me to proceed with this plan?
---------- ThoughtEvent (WebSearchAgent) ----------
Yes, please proceed with the plan. I'll start by finding the Miami Heat player with the highest points in the 2006-2007 NBA season.
---

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=152, completion_tokens=136), metadata={}, content="Here's the plan to answer your question:\n\n1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 NBA season\n2. WebSearchAgent : Find the total rebounds for this player in the 2007-2008 season\n3. WebSearchAgent : Find the total rebounds for this player in the 2008-2009 season\n4. DataAnalystAgent : Calculate the percentage change in rebounds between 2007-2008 and 2008-2009 seasons\n\nOnce we have all this information, I'll summarize the findings and terminate. \n\nWould you like me to proceed with this plan?", type='TextMessage'), Thought

# User Feedback

In [ ]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
Here's the plan to answer your question:

1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season
2. WebSearchAgent : Find the total rebounds for this player in the 2007-2008 season
3. WebSearchAgent : Find the total rebounds for this player in the 2008-2009 season
4. DataAnalystAgent : Calculate the percentage change in rebounds between the 2007-2008 and 2008-2009 seasons

Once we have all this information, I'll compile the final answer and terminate.
Enter your response: APPROVE
---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_0_e7f32a11-7f3e-4a70-adc3-6a384ae365d5', arguments='{"query":

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=152, completion_tokens=126), metadata={}, content="Here's the plan to answer your question:\n\n1. WebSearchAgent : Find the Miami Heat player with the highest points in the 2006-2007 season\n2. WebSearchAgent : Find the total rebounds for this player in the 2007-2008 season\n3. WebSearchAgent : Find the total rebounds for this player in the 2008-2009 season\n4. DataAnalystAgent : Calculate the percentage change in rebounds between the 2007-2008 and 2008-2009 seasons\n\nOnce we have all this information, I'll compile the final answer and terminate.", type='TextMessage'), UserInputRequestedEvent(source='UserProxyAgent', model

# Using Reasoning Models

In [ ]:
model_client = OpenAIChatCompletionClient(
        model="deepseek-reasoner",
        temperature=0.85,
        api_key=userdata.get('DEEPSEEK_API_KEY'),
        base_url="https://api.deepseek.com/v1",
        model_info={
            "model_name": "deepseek-reasoner",
            "api_type": "openai",
            "vendor": "deepseek",
            "json_output": False,
            "vision": False,
            "function_calling": True,
            "function_call": False,
            "streaming": True,
            "cost_per_token": 0,
            "family": "deepseek",
        }
    )

/usr/local/lib/python3.11/dist-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [ ]:
#model_client = OpenAIChatCompletionClient(model="o3-mini",api_key=userdata.get("OPENAI_API_KEY"))

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""Use web search tool to find information.""",
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""Use tool to perform calculation. If you have not seen the data, ask for it.""",
)

user_proxy_agent = UserProxyAgent(
    "UserProxyAgent",
    description="A user to approve or disapprove tasks.",
)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
"""

team = SelectorGroupChat(
    [web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,  # Use the same termination condition as before.
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)


In [ ]:
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ThoughtEvent (WebSearchAgent) ----------
I'll help you find the answer to your question by breaking it down into steps. Let's start by identifying the Miami Heat player with the highest points in the 2006-2007 season.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_0_916c6066-74e2-45af-b8df-1a380e34f44c', arguments='{"query": "Miami Heat 2006-2007 season leading scorer"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_web_tool', call

---------- ThoughtEvent (WebSearchAgent) ----------
We have the following information:
 - For the 2006-2007 season, Dwyane Wade was the leading scorer for the Miami Heat with 1397 points.
 - For the 2007-2008 season, Dwyane Wade had 214 total rebounds.
 - For the 2008-2009 season, Dwyane Wade had 398 total rebounds.

 Now, we need to calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons.

 The formula for percentage change is:
   Percentage Change = [(New Value - Old Value) / Old Value] * 100

 Here:
   Old Value = 214 (2007-2008 season)
   New Value = 398 (2008-2009 season)

 Let's compute:
   Change = 398 - 214 = 184
   Percentage Change = (184 / 214) * 100

 Calculating:
   184 / 214 ≈ 0.8598
   0.8598 * 100 ≈ 85.98%

 Therefore, the percentage change is approximately 85.98%.

 Answer:
   The player with the highest points in the 2006-2007 season for the Miami Heat was Dwyane Wade.
   The percentage change in his total rebounds from the 20